In [12]:
from datetime import datetime, timedelta
import requests
from datetime import datetime
import pandas as pd
from dotenv import main
import os

In [15]:
main.load_dotenv()
api_key = os.getenv("API_KEY")
username = "write_username_here"

In [4]:
class LastFMAPI:
    WEEK_DIFF = 604800
    FILE_NAME = "alltracks.csv"
    def __init__(self, username, key, limit=1000):
        self.username = username
        self.key = key
        self.limit = limit
        self.start_date = datetime(year=2022, month=12, day=1)  # first time using lastfm
        self.start_utc = int(self.start_date.timestamp()) # utc
        self.dfs = []
        self.current_time = datetime.now()
        self.utc_time = int(self.current_time.timestamp())
        self.df = pd.DataFrame()
        
    def get_start_date(self):
        return self.start_utc
    
    def set_next_start_date(self):
        if self.start_utc + LastFMAPI.WEEK_DIFF > self.utc_time:
            self.start_utc = self.utc_time
        else:
            self.start_utc += LastFMAPI.WEEK_DIFF
        
    def get_end_date(self):
        return self.start_utc + LastFMAPI.WEEK_DIFF
    
    def get_tags(self, artists):  # only alphanumeric
        tags = {}
        for artist in (set(artists)):
            url = f"https://ws.audioscrobbler.com/2.0/?method=artist.getTopTags&artist={artist}&api_key={self.key}&format=json"
            res = requests.get(url).json()
            print(res)
            if 'toptags' in res and 'tag' in res['toptags'] and len(res['toptags']['tag']) > 0:
                top = res['toptags']['tag'][0]['name']
                tags[artist] = top
            else:
                print(f"Error or no tags found for artist: {artist}")
        return tags
            
        
        
    def get_and_save_all_tags(self):
        self.df['tags'] = self.get_tags_vectorized(self.df['artist'])
        self.df.to_csv(LastFMAPI.FILE_NAME, encoding="utf-8")

    def get_tags_vectorized(self, artists):
        tags_dict = self.get_tags(artists)
        return artists.map(lambda artist: tags_dict.get(artist, []))

        
    def save_tracks(self):
        url = f"https://ws.audioscrobbler.com/2.0/?api_key={self.key}&method=user.getrecenttracks&user={self.username}&from={self.get_start_date()}&to={self.get_end_date()}&limit={self.limit}&format=json"
        res = requests.get(url).json()
        tracks = res['recenttracks']["track"]
        track_name = []
        artist = []
        album_name = []
        date_listened = []
        uts = []
        urls = []
        for track in tracks:  # might be more efficient to build the dataframe by row, instead of column.
            try:
                artist.append(track['artist']['#text'])
            except KeyError:
                artist.append("NA")
            
            try:
                album_name.append(track['album']['#text'])
            except KeyError:
                album_name.append("NA")
            
            try:
                track_name.append(track['name'])
            except KeyError:
                track_name.append("NA")
            
            try:
                date_listened.append(track['date']['#text'])
            except KeyError:
                date_listened.append("NA")
            
            try:
                uts.append(track['date']['uts'])
            except KeyError:
                uts.append("NA")
            
            try:
                urls.append(track['url'])
            except KeyError:
                urls.append("NA")
            
        df = pd.DataFrame({
            'track_name': track_name,
            'artist': artist,
            'album_name': album_name,
            'date_listened': date_listened,
            'uts': uts,
            'url': urls,
        })
        self.dfs.append(df)
    def get_csv_file(self):
        merged_table = pd.concat(self.dfs, ignore_index=True)
        self.df = merged_table
        merged_table.to_csv(LastFMAPI.FILE_NAME, encoding="utf-8")
    
    def get_all_tracks(self):
        while self.get_start_date() < self.utc_time:
            self.save_tracks()
            self.set_next_start_date() 
            print("Saving..")
        print("DONE!")
        

In [5]:
lastfmapi = LastFMAPI(username, api_key)
lastfmapi.get_all_tracks()
lastfmapi.get_csv_file()


Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
Saving..
DONE!


In [6]:
lastfmapi.get_and_save_all_tags()


{'toptags': {'tag': [{'count': 100, 'name': 'folk', 'url': 'https://www.last.fm/tag/folk'}, {'count': 90, 'name': 'Ukrainian', 'url': 'https://www.last.fm/tag/Ukrainian'}, {'count': 48, 'name': 'ethno-chaos', 'url': 'https://www.last.fm/tag/ethno-chaos'}, {'count': 30, 'name': 'seen live', 'url': 'https://www.last.fm/tag/seen+live'}, {'count': 23, 'name': 'ethnochaos', 'url': 'https://www.last.fm/tag/ethnochaos'}, {'count': 22, 'name': 'ethno chaos', 'url': 'https://www.last.fm/tag/ethno+chaos'}, {'count': 15, 'name': 'ethno', 'url': 'https://www.last.fm/tag/ethno'}, {'count': 13, 'name': 'Ukraine', 'url': 'https://www.last.fm/tag/Ukraine'}, {'count': 12, 'name': 'ethnic', 'url': 'https://www.last.fm/tag/ethnic'}, {'count': 8, 'name': 'World Music', 'url': 'https://www.last.fm/tag/World+Music'}, {'count': 6, 'name': 'world', 'url': 'https://www.last.fm/tag/world'}, {'count': 5, 'name': 'neofolk', 'url': 'https://www.last.fm/tag/neofolk'}, {'count': 5, 'name': 'electronic folk', 'url': 

In [7]:
df = pd.read_csv(LastFMAPI.FILE_NAME)
df.head()

,Unnamed: 0,track_name,artist,album_name,date_listened,uts,url,tags
0,0,Untouched,SilenCee,Untouched,"07 Dec 2022, 09:30",1670405412,https://www.last.fm/music/SilenCee/_/Untouched,electronic
1,1,maria,Malte,maria,"07 Dec 2022, 09:18",1670404731,https://www.last.fm/music/Malte/_/maria,german
2,2,let me go,Malte,let me go,"07 Dec 2022, 09:16",1670404596,https://www.last.fm/music/Malte/_/let+me+go,german
3,3,So What,mqx,So What,"07 Dec 2022, 09:14",1670404473,https://www.last.fm/music/mqx/_/So+What,hardstyle
4,4,maria,Malte,maria,"07 Dec 2022, 09:11",1670404288,https://www.last.fm/music/Malte/_/maria,german


In [8]:
df.tail()

,Unnamed: 0,track_name,artist,album_name,date_listened,uts,url,tags
18553,18553,Slippery People - Live,Talking Heads,Stop Making Sense (Live),"07 Aug 2024, 21:15",1723065315,https://www.last.fm/music/Talking+Heads/_/Slip...,new wave
18554,18554,Found a Job,Talking Heads,Stop Making Sense (Live),"07 Aug 2024, 21:12",1723065120,https://www.last.fm/music/Talking+Heads/_/Foun...,new wave
18555,18555,Thank You for Sending Me an Angel - Live,Talking Heads,Stop Making Sense (Live),"07 Aug 2024, 21:09",1723064990,https://www.last.fm/music/Talking+Heads/_/Than...,new wave
18556,18556,Heaven - Live,Talking Heads,Stop Making Sense (Live),"07 Aug 2024, 21:06",1723064770,https://www.last.fm/music/Talking+Heads/_/Heav...,new wave
18557,18557,Psycho Killer - live,Talking Heads,Stop Making Sense (Live),"07 Aug 2024, 21:01",1723064494,https://www.last.fm/music/Talking+Heads/_/Psyc...,new wave
